<a href="https://colab.research.google.com/github/tsuli/LSC_project/blob/main/Task2_v04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'

# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip

# unzip the file
!unzip -qq UCI_HAR_Dataset.zip

# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.1M  100 58.1M    0     0  88.9M      0 --:--:-- --:--:-- --:--:-- 88.8M


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt').astype(np.float32) -1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt').astype(np.float32) -1

# load the x,y,z body accelerations test data
xx=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt').astype(np.float32) 

# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)

# follow the same approach for the train data
xx=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt').astype(np.float32) 
yy=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt').astype(np.float32) 
zz=np.loadtxt('/content/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt').astype(np.float32) 

xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)

xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)
ytrain_enc = tf.keras.utils.to_categorical(ytrain)
yvalid_enc = tf.keras.utils.to_categorical(yvalid)
ytest_enc = tf.keras.utils.to_categorical(ytest)
xtrain = xtrain.astype(np.float32)
xvalid = xvalid.astype(np.float32)
xtest = xtest.astype(np.float32)

print(xtrain.shape, ytrain_enc.shape, xvalid.shape, yvalid_enc.shape, xtest.shape, ytest_enc.shape)

(5881, 128, 3) (5881, 6) (1471, 128, 3) (1471, 6) (2947, 128, 3) (2947, 6)


In [85]:
input_layer = tf.keras.Input(shape=(128,3))
output_layer1 = tf.keras.layers.Conv1D(filters=32, kernel_size=(4))
output_layer2 = tf.keras.layers.BatchNormalization()
output_layer3 = tf.keras.layers.ReLU()
output_layer4 = tf.keras.layers.AveragePooling1D()
output_layer5 = tf.keras.layers.Dense(units=6,activation=tf.nn.softmax)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [79]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
model.fit(x_input, y_input, epochs=100, batch_size=256, validation_data=(x_valid, y_valid))